## Load Modules

In [1]:
import word2vecmodel
import preprocess
import engine
import visualize
import pandas as pd

## 태그 임베딩 모델 불러오기

In [2]:
enc = engine.load("save")

# 변수 초기화

In [7]:
top_k = 40
jobkeywordlist = preprocess.getJobList()

## 테스트

In [8]:
usertaglist = ['HTML', 'CSS', "JavaScript", 'jQuery']
engine.cal_score(usertaglist, jobkeywordlist, top_k)

,jobID,tags,score
0,5a4b25aaed25c2ee1b74de72dc03c14e,"[CSS, HTML, jQuery, JavaScript]",1.000000
1,286674e3082feb7e5afb92777e48821f,"[HTML, JavaScript, CSS]",1.000000
2,2b6d65b9a9445c4271ab9076ead5605a,"[JavaScript, CSS, HTML]",1.000000
3,63538fe6ef330c13a05a3ed7e599d5f7,"[CSS, JavaScript, HTML]",1.000000
4,84d2004bf28a2095230e8e14993d398d,"[JavaScript, CSS, HTML]",1.000000
5,995665640dc319973d3173a74a03860c,"[JavaScript, CSS, HTML]",1.000000
6,c042f4db68f23406c6cecf84a7ebb0fe,"[HTML, CSS, JavaScript]",1.000000
7,dc912a253d1e9ba40e2c597ed2376640,"[CSS, JavaScript, HTML]",1.000000
8,fb89705ae6d743bf1e848c206e16a1d7,"[JavaScript, HTML, CSS]",1.000000
9,ebd9629fc3ae5e9f6611e2ee05a31cef,"[JavaScript, HTML]",1.000000


## 테스트 with GUI

In [17]:
import ipywidgets as widgets

desc = widgets.Label(value='기술 스택 (쉼표로 구분)')
tags_input = widgets.Text(
    value='',
    placeholder='예: HTML, CSS, JavaScript',
    description='',
    disabled=False,
    layout=widgets.Layout(width='50%')
)
search_button = widgets.Button(
    description='검색',
    layout=widgets.Layout(width='10%'))
input_with_button = widgets.HBox([tags_input, search_button])
output_html = widgets.HTML(
    value=""
)
display(widgets.VBox([desc, input_with_button, output_html]))

def search(b):
    tags_input_ed = [x.strip() for x in tags_input.value.split(',')]
    styles = [
        {'selector':'td', 'props':[('padding', '10px'), ('vertical-align', 'top'), ('border-bottom', '1px solid #ccc'), ('background', '#eee')
         ]}, 
        {'selector':'th', 'props':[('padding', '10px'), ('font-weight', 'bold'), ('vertical-align', 'top'), ('color', '#fff'), ('background', '#263747')]}
    ]
    output_html.value = engine.cal_score(tags_input_ed, jobkeywordlist, top_k).style.set_table_styles(styles).render()

search_button.on_click(search)